# Jira Development Status Checker

This notebook checks whether Jira tickets have associated development activities (commits, branches, pull requests) using the Jira Development Status API.

## Usage

1. Update the configuration variables below
2. Create a `jira_links.txt` file with one Jira ticket URL per line
3. Run all cells
4. Enter your Jira API token when prompted

In [ ]:
import requests
import re
import getpass

# Configuration
JIRA_DOMAIN = "your-org.atlassian.net"
JIRA_EMAIL = "email@email.com"
JIRA_API_TOKEN = getpass.getpass("Enter your Jira API Token: ").strip()

In [ ]:
AUTH = (JIRA_EMAIL, JIRA_API_TOKEN)

HEADERS = {
    "Accept": "application/json"
}

ISSUE_API_URL = f"https://{JIRA_DOMAIN}/rest/api/3/issue"
DEV_STATUS_API = f"https://{JIRA_DOMAIN}/rest/dev-status/1.0/issue/details"

def extract_issue_key(url):
    """Extract Jira issue key from URL."""
    match = re.search(r'/browse/([A-Z]+-\d+)', url)
    return match.group(1) if match else None

def get_issue_id(issue_key):
    """Get internal issue ID for Jira issue key."""
    url = f"{ISSUE_API_URL}/{issue_key}"
    response = requests.get(url, auth=AUTH, headers=HEADERS)
    if response.status_code == 200:
        return response.json().get("id")
    else:
        print(f"Error fetching issue ID for {issue_key}: {response.status_code}")
        return None

def check_development_info(issue_id):
    """Check if development data exists for a given issue ID."""
    params = {
        "issueId": issue_id
    }
    response = requests.get(DEV_STATUS_API, headers=HEADERS, auth=AUTH, params=params)
    if response.status_code == 200:
        details = response.json().get("detail", [])
        total = sum(len(item.get("repositories", [])) for item in details)
        return total > 0
    else:
        print(f"Error fetching development info for issue ID {issue_id}: {response.status_code}")
        return False

def main():
    with open("jira_links.txt", "r") as file:
        for line in file:
            url = line.strip()
            issue_key = extract_issue_key(url)
            if not issue_key:
                print(f"Invalid URL: {url}")
                continue

            issue_id = get_issue_id(issue_key)
            if issue_id:
                has_dev = check_development_info(issue_id)
                print(f"{issue_key}: {'HAS development' if has_dev else 'NO development'}")
            else:
                print(f"{issue_key}: Could not fetch issue ID")

if __name__ == "__main__":
    main()

## Example Output

```
ABC-123: HAS development
DEF-456: NO development
GHI-789: Could not fetch issue ID
```

## Notes

- This script checks the Jira Development Status API to determine if there are any associated development activities
- Development activities include commits, branches, and pull requests linked to the Jira ticket
- You must have proper permissions to view development information in Jira
- Development information is only available if Jira is integrated with a source control system like GitHub, Bitbucket, or GitLab